In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler,PolynomialFeatures
from sklearn.neighbors import KNeighborsClassifier
import warnings
warnings.filterwarnings('ignore')

C:\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [3]:
np.random.seed(42)

In [4]:
df = pd.read_csv("http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv",sep = ';')


In [14]:
y = df.pop('quality')

In [ ]:
df.head(2)

In [6]:
df['quality'].value_counts()

6    2198
5    1457
7     880
8     175
4     163
3      20
9       5
Name: quality, dtype: int64

In [8]:
df.shape

(4898, 12)

In [9]:
df.isna().sum()

fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64

In [12]:
for i in df.columns:
    df[i] = df[i].fillna(np.mean(df[i]))

In [17]:
train_x,test_x,train_y,test_y = train_test_split(df,y,test_size=0.2)

In [21]:
def model_fit(alg,train_x,test_x,train_y,test_y,if_cv=True,cv_folds=5):
    alg.fit(train_x,train_y)
    
    #crossvalidation
    
    if if_cv:  # set to true
        cv_score = cross_val_score(alg,train_x,train_y,cv = cv_folds,scoring='f1_macro')
        # f1 --- it is harmoonized mean
    prediction = alg.predict(test_x)
    
    if if_cv:
        print("CV Report:  Mean -%.3g | std - %3g | Min - %3g,max - %3g" %(np.mean(cv_score)),
              np.std(cv_score),
              np.min(cv_score),
              np.max(cv_score))
    print("Accuray:",accuracy_score(test_y,prediction))
    print('-'*100)
    
    cm = pd.DataFrame(confusion_matrix(test_y,prediction))
    print(cm)


In [23]:
def model_fit(alg, train_x, test_x, train_y, test_y, if_cv = True, cv_folds = 5):
    alg.fit(train_x, train_y)
    
    #Cross-val
    if if_cv:
        cv_score = cross_val_score(alg, train_x, train_y, cv = cv_folds, scoring = 'f1_macro')
    
    prediction = alg.predict(test_x)
    
    if if_cv:
        print('CV report: Mean - %.3g | Std - %.3g | Min - %.3g, Max- %.3g' %(np.mean(cv_score),
                                                                              np.std(cv_score), 
                                                                              np.min(cv_score), 
                                                                              np.max(cv_score)))
    print("Accuracy: ", accuracy_score(test_y, prediction))
    print('-'*100)
    
    cm = pd.DataFrame(confusion_matrix(test_y, prediction))
    print(cm)


In [24]:
lr = LogisticRegression()
model_fit(lr,train_x,test_x,train_y,test_y)

CV report: Mean - 0.158 | Std - 0.0149 | Min - 0.143, Max- 0.186
Accuracy:  0.4846938775510204
----------------------------------------------------------------------------------------------------
   0  1   2    3  4  5  6
0  0  0   1    3  0  0  0
1  0  0  17   21  1  0  0
2  0  0  94  195  1  0  0
3  0  0  75  372  3  0  0
4  0  0  15  147  9  0  0
5  0  0   3   20  2  0  0
6  0  0   0    1  0  0  0


In [28]:
def KNN_predicts(train_x,test_x,train_y,test_y,scaler,neighbors,metric='Manhattan',weights='uniform'):
    train_scaled = scaler.fit_transform(train_x)
    test_scaled = scaler.fit_transform(test_x)
    
    KNN = KNeighborsClassifier(n_neighbors=neighbors,metric=metric,weights=weights,n_jobs=1)
    KNN.fit(train_scaled,train_y)
    prediction = KNN.predict(test_scaled)
    
    print("Accuracy:" ,accuracy_score(test_y,prediction))
    print('-'*100)
    return KNN

In [36]:
def KNN_predicts(train_x, test_x, train_y, test_y, scaler, neighbours, 
                 metric = 'manhattan', 
                 weights = 'uniform'):
    
    train_scaled = scaler.fit_transform(train_x)
    test_scaled = scaler.transform(test_x)
    
    KNN = KNeighborsClassifier(n_neighbors= neighbours, metric = metric, 
                               weights = weights, n_jobs = -1)
    KNN.fit(train_scaled, train_y)
    prediction = KNN.predict(test_scaled)
    
    print('Accuracy: ', accuracy_score(test_y, prediction))
    print('-'*100)
    return KNN


In [37]:
KNN_predicts(train_x,test_x,train_y,test_y,StandardScaler(),1)

Accuracy:  0.636734693877551
----------------------------------------------------------------------------------------------------


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='manhattan',
                     metric_params=None, n_jobs=-1, n_neighbors=1, p=2,
                     weights='uniform')

# Neighbor tuning

In [39]:
for k in range(1,11):
    print("Accuracy score on KNN using N-neighours = {0}:" .format(2**k),end = '')
    KNN_predicts(train_x,test_x,train_y,test_y,StandardScaler(),2**k)

Accuracy score on KNN using N-neighours = 2:Accuracy:  0.5663265306122449
----------------------------------------------------------------------------------------------------
Accuracy score on KNN using N-neighours = 4:Accuracy:  0.5489795918367347
----------------------------------------------------------------------------------------------------
Accuracy score on KNN using N-neighours = 8:Accuracy:  0.5551020408163265
----------------------------------------------------------------------------------------------------
Accuracy score on KNN using N-neighours = 16:Accuracy:  0.5408163265306123
----------------------------------------------------------------------------------------------------
Accuracy score on KNN using N-neighours = 32:Accuracy:  0.5183673469387755
----------------------------------------------------------------------------------------------------
Accuracy score on KNN using N-neighours = 64:Accuracy:  0.5295918367346939
------------------------------------------------

In [42]:
k =5

for metric in ['euclidean', 'minkowski', 'manhattan', 'chebyshev']:
    print('Accuracy score on KNN using {} metric and  {} neighbors :'.format(metric, k), end = ' ')
    KNN_predicts(train_x, test_x, train_y, test_y, StandardScaler(), k, metric)


Accuracy score on KNN using euclidean metric and  5 neighbors : Accuracy:  0.5581632653061225
----------------------------------------------------------------------------------------------------
Accuracy score on KNN using minkowski metric and  5 neighbors : Accuracy:  0.5581632653061225
----------------------------------------------------------------------------------------------------
Accuracy score on KNN using manhattan metric and  5 neighbors : Accuracy:  0.5510204081632653
----------------------------------------------------------------------------------------------------
Accuracy score on KNN using chebyshev metric and  5 neighbors : Accuracy:  0.5428571428571428
----------------------------------------------------------------------------------------------------


# Model Pickle

In [43]:
KNN

NameError: name 'KNN' is not defined

In [45]:
import joblib
joblib.dump(lr,'KNN_MODEL_RAJ.pkl')

['KNN_MODEL_RAJ.pkl']